# 🏎️ Formula 1 Tire Change Prediction - Quick Start (Colab Pro)

**Sistema completo di predizione cambi gomme F1 con RNN multi-task**

Questo notebook ti guida attraverso il setup completo e il training del modello ottimizzato per Google Colab Pro.

## 📋 Pre-requisiti
- Google Colab Pro (raccomandato)
- Dati F1 caricati su Google Drive
- GPU abilitata (Runtime > Change runtime type > GPU)

---

## 🚀 Step 1: Setup Ambiente

Iniziamo con il setup automatico dell'ambiente ottimizzato per Colab Pro.

In [ ]:
# Monta Google Drive all'inizio per accesso ai dati e salvataggio modelli
from google.colab import drive
import os

if not os.path.exists('/content/drive'):
  print("📂 Montaggio Google Drive...")
  drive.mount('/content/drive')
else:
  print("✅ Google Drive già montato.")

In [ ]:
!git clone https://github.com/Vladinsky/FASTF1.git
%cd FASTF1/colab_models

In [ ]:
# Clona repository (se necessario) o carica files
# NOTA: Se stai usando questo notebook, presumiamo che i file siano già caricati

import os
import sys

# Verifica che siamo nella directory corretta
if os.getcwd().endswith('FASTF1/colab_models'):
    print("✅ Già nella directory colab_models!")
elif os.path.exists('colab_models'): # Se siamo in /content/FASTF1
    print("🔄 Cambio directory in colab_models...")
    %cd colab_models
elif os.path.exists('FASTF1/colab_models'): # Se siamo in /content
    print("🔄 Cambio directory in FASTF1/colab_models...")
    %cd FASTF1/colab_models
else:
    print("❌ Directory colab_models non trovata! Assicurati che il git clone sia avvenuto correttamente.")
    print(f"📍 Directory attuale: {os.getcwd()}")

print(f"✅ Directory di lavoro corrente: {os.getcwd()}")

In [ ]:
# Esegui setup automatico
print("🚀 Avvio setup automatico per Colab Pro...")
if os.path.exists('setup_colab_pro.py'):
    %run setup_colab_pro.py
else:
    print("❌ File setup_colab_pro.py non trovato! Verifica la directory.")

## 📊 Step 2: Unificazione Dati

I dati F1 sono attualmente distribuiti in multiple cartelle. Li unifichiamo in un dataset completo.

In [ ]:
# Verifica dati disponibili su Drive
import os

# Assicurati che Drive sia montato (già fatto sopra, ma un controllo non fa male)
if not os.path.exists('/content/drive/MyDrive'):
    print("❌ Google Drive non sembra montato correttamente. Riprova il montaggio.")
else:
    print("✅ Google Drive è montato.")

data_path = "/content/drive/MyDrive/F1_Project/processed_races"

print("📁 Controllo dati disponibili:")
if os.path.exists(data_path):
    try:
        num_files = len(os.listdir(data_path))
        print(f"Percorso dati: {data_path} - Trovato ({num_files} files)")
    except Exception as e:
        print(f"⚠️ Errore nel leggere la directory {data_path}: {e}")
else:
    print(f"❌ Percorso dati: {data_path} - NON Trovato!")
    print("⚠️  Assicurati di aver caricato i dati su Drive nel percorso corretto e che Drive sia montato.")

In [ ]:
# Unifica tutti i dati in un dataset completo
from data.data_unifier_complete import CompleteDataUnifier

print("🔄 Avvio unificazione dati...")
print("⏱️  Questo processo può richiedere 10-15 minuti")

unifier = CompleteDataUnifier(config_path="colab_models/configs/data_config_colab.json")
dataset = unifier.unify_all_data()

if dataset is not None and not dataset.empty:
    print(f'✅ Unificazione completata!')
    print(f'📊 Dataset finale: {len(dataset):,} righe, {len(dataset.columns)} colonne')
else:
    print(f'❌ Unificazione fallita o dataset vuoto.')

In [ ]:
# Esplorazione rapida del dataset unificato
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

if 'dataset' in locals() and dataset is not None and not dataset.empty:
    print("🔍 Esplorazione dataset unificato:")
    print(f'Shape: {dataset.shape}')
    print(f'Anni: {sorted(dataset["Year"].unique()) if "Year" in dataset.columns else "N/A"}')
    print(f'Piloti: {dataset["Driver"].nunique() if "Driver" in dataset.columns else "N/A"}')
    print(f'GP: {dataset["EventName"].nunique() if "EventName" in dataset.columns else "N/A"}')

    # Visualizzazione distribuzione anni
    if 'Year' in dataset.columns:
        plt.figure(figsize=(10, 6))
        dataset['Year'].value_counts().sort_index().plot(kind='bar')
        plt.title('Distribuzione Dati per Anno')
        plt.xlabel('Anno')
        plt.ylabel('Numero di Record')
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.show()
else:
    print("❌ Dataset non disponibile per l'esplorazione.")

## 🏋️ Step 3: Training del Modello

Avviamo il training completo del modello LSTM multi-task ottimizzato per Colab Pro.

In [ ]:
# Verifica GPU e memoria
import torch
import psutil

print("🔧 Controllo risorse sistema:")

# GPU
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'✅ GPU: {gpu_name} ({gpu_memory:.1f}GB)')
    
    if "T4" in gpu_name or "A100" in gpu_name or "V100" in gpu_name:
        print("🚀 GPU ottima per training!")
    else:
        print("⚠️  GPU rilevata ma potrebbe essere lenta")
else:
    print("❌ GPU non disponibile! Abilita GPU in Runtime > Change runtime type")

# RAM
memory = psutil.virtual_memory()
total_gb = memory.total / 1e9
available_gb = memory.available / 1e9

print(f'✅ RAM: {total_gb:.1f}GB totale, {available_gb:.1f}GB disponibile')

if total_gb > 20:
    print("🚀 RAM eccellente per Colab Pro!")
elif total_gb > 12:
    print("✅ RAM buona per training")
else:
    print("⚠️  RAM limitata, considera Colab Pro")

In [ ]:
# Setup trainer
from training.train_from_scratch_pro import ProTrainer

print("🏋️ Inizializzazione trainer...")

# Crea trainer con configurazione ottimizzata
# Assicurati che il project_dir sia corretto e accessibile da Colab (su Drive)
project_drive_dir = "/content/drive/MyDrive/F1_TireChange_Project"
if not os.path.exists(project_drive_dir):
    print(f"⚠️ Directory progetto {project_drive_dir} non trovata su Drive. Verrà creata.")
    os.makedirs(project_drive_dir, exist_ok=True)
    os.makedirs(os.path.join(project_drive_dir, 'models/checkpoints'), exist_ok=True)
    os.makedirs(os.path.join(project_drive_dir, 'results'), exist_ok=True)
    print(f"✅ Directory progetto {project_drive_dir} creata.")

trainer = ProTrainer(
    config_path="configs/model_config_pro.yaml",
    project_dir=project_drive_dir
)

print("✅ Trainer inizializzato!")
print(f'📱 Device: {trainer.device}')
print(f'📁 Model dir: {trainer.model_dir}')
print(f'📊 Results dir: {trainer.results_dir}')

In [ ]:
# AVVIO TRAINING COMPLETO
print("🚀 AVVIO TRAINING DA ZERO...")
print("⏱️  Tempo stimato: 4-6 ore su Colab Pro")
print("💾 Checkpoint automatici ogni 30 minuti")
print("🔄 Puoi interrompere e riprendere il training")
print("="*60)

# Training con parametri ottimizzati
if 'trainer' in locals():
    results = trainer.train_complete(max_epochs=100)

    print("="*60)
    print("🎉 TRAINING COMPLETATO!")
    print(f'⏱️  Tempo totale: {results["total_training_time"]/3600:.1f} ore')
    print(f'🎯 Best validation loss: {results["best_val_loss"]:.4f}')
else:
    print("❌ Trainer non inizializzato. Impossibile avviare il training.")

## 📊 Step 4: Analisi Risultati

Analizziamo i risultati del training e le performance del modello.

In [ ]:
# Visualizza curve di training
import matplotlib.pyplot as plt
import numpy as np

if 'results' in locals() and results:
    history = results['training_history']

    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Analisi Risultati Training', fontsize=16)

    # Loss curves
    axes[0,0].plot(history['epochs'], history['train_loss'], label='Train Loss', color='blue')
    axes[0,0].plot(history['epochs'], history['val_loss'], label='Val Loss', color='red')
    axes[0,0].set_title('Training & Validation Loss')
    axes[0,0].set_xlabel('Epoch')
    axes[0,0].set_ylabel('Loss')
    axes[0,0].legend()
    axes[0,0].grid(True)

    # Accuracy curves
    axes[0,1].plot(history['epochs'], history['train_acc'], label='Train Acc', color='blue')
    axes[0,1].plot(history['epochs'], history['val_acc'], label='Val Acc', color='red')
    axes[0,1].set_title('Training & Validation Accuracy')
    axes[0,1].set_xlabel('Epoch')
    axes[0,1].set_ylabel('Accuracy')
    axes[0,1].legend()
    axes[0,1].grid(True)

    # Learning rate
    if 'learning_rates' in history:
        axes[1,0].plot(history['epochs'], history['learning_rates'], color='green')
        axes[1,0].set_title('Learning Rate Schedule')
        axes[1,0].set_xlabel('Epoch')
        axes[1,0].set_ylabel('Learning Rate')
        axes[1,0].set_yscale('log')
        axes[1,0].grid(True)
    else:
        axes[1,0].text(0.5, 0.5, 'Learning rate data non disponibile',
                       horizontalalignment='center', verticalalignment='center',
                       transform=axes[1,0].transAxes)
        axes[1,0].set_title('Learning Rate Schedule')

    # Final metrics
    if 'final_test_metrics' in results:
        test_metrics = results['final_test_metrics']
        metrics_names = list(test_metrics.keys())
        metrics_values = list(test_metrics.values())
        axes[1,1].bar(metrics_names, metrics_values, color='skyblue')
        axes[1,1].set_title('Final Test Metrics')
        axes[1,1].set_ylabel('Score')
        axes[1,1].tick_params(axis='x', rotation=45)
        axes[1,1].grid(axis='y')
    else:
        axes[1,1].text(0.5, 0.5, 'Metriche finali non disponibili',
                       horizontalalignment='center', verticalalignment='center',
                       transform=axes[1,1].transAxes)
        axes[1,1].set_title('Final Test Metrics')

    plt.tight_layout(rect=[0, 0, 1, 0.96]) # Aggiusta layout per suptitle
    plt.show()

    if 'final_test_metrics' in results:
        print("📊 Risultati Finali Test:")
        for metric, value in results['final_test_metrics'].items():
            print(f'  {metric}: {value:.4f}')
else:
    print("❌ Risultati del training non disponibili per l'analisi.")

In [ ]:
# Informazioni modello finale
if 'trainer' in locals() and hasattr(trainer, 'model') and 'results' in locals() and results:
    model_info = trainer.model.get_model_info()

    print("🧠 Informazioni Modello:")
    print(f'  Parametri totali: {model_info["total_parameters"]:,}')
    print(f'  Parametri LSTM: {model_info["lstm_parameters"]:,}')
    print(f'  Parametri Shared Trunk: {model_info["shared_trunk_parameters"]:,}')
    print(f'  Input shape: {model_info["input_shape"]}')
    print(f'  Hidden size: {model_info["hidden_size"]}')
    print(f'  Num layers: {model_info["num_layers"]}')

    # Calcola dimensione modello in MB
    model_size_mb = model_info['total_parameters'] * 4 / 1024 / 1024  # 4 bytes per parameter
    print(f'  Dimensione modello: {model_size_mb:.1f} MB')
else:
    print("❌ Informazioni modello non disponibili (trainer o modello non inizializzati, o training non eseguito).")

## 🚀 Step 5: Test Inference (Opzionale)

Testiamo il modello addestrato con alcune predizioni di esempio.

In [ ]:
# Test rapido inference
import torch
import numpy as np
import os

if 'trainer' in locals() and hasattr(trainer, 'model'):
    # Carica best model
    best_model_path = os.path.join(trainer.model_dir, "checkpoints/best_model.pth")

    if os.path.exists(best_model_path):
        print(f"🔄 Caricamento best model da: {best_model_path}...")
        
        try:
            checkpoint = torch.load(best_model_path, map_location=trainer.device)
            trainer.model.load_state_dict(checkpoint['model_state_dict'])
            trainer.model.eval()
            print("✅ Modello caricato e impostato in modalità eval")
            
            # Crea un batch di esempio per test
            with torch.no_grad():
                # Input di esempio (sostituire con dati reali)
                batch_size = 4
                sequence_length = trainer.config['data_loader']['sequence_length'] # Usa sequence_length da config
                input_features = trainer.model.input_size
                
                sample_input = torch.randn(batch_size, sequence_length, input_features).to(trainer.device)
                
                print(f"🔍 Testing con input shape: {sample_input.shape}")
                
                # Predizione
                outputs = trainer.model(sample_input)
                
                print("📊 Output del modello:")
                for task_name, output in outputs.items():
                    print(f"  {task_name}: shape {output.shape}")
                    if 'classification' in task_name.lower() or 'pit_stop_prediction' in task_name.lower():
                        probs = torch.softmax(output, dim=-1)
                        predictions = torch.argmax(probs, dim=-1)
                        print(f"    Predictions: {predictions.cpu().numpy()}")
                        print(f"    Max probabilities: {torch.max(probs, dim=-1)[0].cpu().numpy()}")
                    else: # Regression task
                        print(f"    Values (first 10): {output.cpu().numpy().flatten()[:10]}...")
                
                print("✅ Test inference completato con successo!")
        except Exception as e:
            print(f"❌ Errore durante il caricamento del modello o l'inference: {e}")
            
    else:
        print(f"❌ Best model non trovato in {best_model_path}. Completa prima il training.")
else:
    print("❌ Trainer o modello non inizializzati. Impossibile eseguire l'inference.")

In [ ]:
# Salva un riassunto finale
import json
from datetime import datetime
import os

if 'results' in locals() and results and 'model_info' in locals() and model_info and 'dataset' in locals() and dataset is not None:
    summary = {
        'timestamp': datetime.now().isoformat(),
        'dataset_info': {
            'total_rows': len(dataset),
            'total_columns': len(dataset.columns),
            'years_covered': sorted(dataset['Year'].unique().tolist()) if 'Year' in dataset.columns else [],
            'unique_drivers': dataset['Driver'].nunique() if 'Driver' in dataset.columns else 0,
            'unique_events': dataset['EventName'].nunique() if 'EventName' in dataset.columns else 0
        },
        'training_results': results,
        'model_info': model_info,
        'gpu_info': {
            'device_name': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU',
            'memory_gb': torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0
        }
    }

    summary_path = os.path.join(trainer.project_dir, "training_summary.json") # Salva in project_dir su Drive
    try:
        with open(summary_path, 'w') as f:
            json.dump(summary, f, indent=2, default=str) # default=str per gestire tipi non serializzabili come numpy int64
        print(f"📋 Riassunto salvato in: {summary_path}")
        print("🎉 Training completo! Il modello è pronto per l'uso.")
    except Exception as e:
        print(f"❌ Errore durante il salvataggio del riassunto: {e}")
else:
    print("❌ Dati mancanti per salvare il riassunto (results, model_info, o dataset non disponibili).")

In [ ]:
# Test con dati specifici (pilota, anno, circuito)
import subprocess
import os

print("🧪 Avvio test con dati specifici...")

# Parametri test
driver = "VER"  # Sostituisci con il codice del pilota
year = 2024      # Anno della gara
circuit = "Bahrain"  # Nome del circuito (come nel file)

# Esegui script test_model_new_data.py
# Assicurati che lo script sia nella directory corretta o fornisci il path completo
script_path = "inference/test_model_new_data.py" 
if not os.path.exists(script_path):
    print(f"❌ Script {script_path} non trovato. Verifica il path.")
else:
    command = f"python {script_path} --driver '{driver}' --year {year} --circuit '{circuit}'"
    print(f"🔧 Esecuzione comando: {command}")
    try:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate()

        print("Output:")
        print(stdout)
        if stderr:
            print("Errori (se presenti):")
            print(stderr)
        print("✅ Test con dati specifici completato.")
    except Exception as e:
        print(f"❌ Errore durante l'esecuzione dello script di test: {e}")